In [1]:
def check_high_iwp(iwp_mean):
    
    a = iwp_mean > 120
    if np.sum(a) > 0:
        return True
    else:
        return False
    
import xarray
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from iwc2tb.GMI.GMI_SatData import GMI_Sat
from iwc2tb.GMI.gmiSatData import gmiSatData
import pickle
import time
#time.sleep(60 * 60 * 3)


#---------------------------------
year  = "2020"
month = "01"
#---------------------------------

path = os.path.expanduser("~/Dendrite/UserAreas/Kaur/IWP/lpa_pr1")
files = glob.glob(os.path.join(path, "1B*202001*nc"))

gpath = os.path.expanduser(os.path.join("~/Dendrite/SatData/GMI/L1B/", year, month))
gfiles = glob.glob(os.path.join(gpath, "*", "*HDF5"))

inputs             = ["ta", "t2m",  "wvp", "lat","z0", "stype"]
outputs            = "iwp"
batchSize          = 4
latlims            = [0, 65]


LAT = np.zeros([1, 221])
LON = np.zeros([1, 221])
IWP = np.zeros([1, 221])
IWP_mean = np.zeros([1, 221])
LSM = np.zeros([1, 221])
IWP0 = np.zeros([1, 221])
Z0 = np.zeros([1, 221])
WVP = np.zeros([1, 221])
TB   = np.zeros([1, 221, 4])

for ix, file in enumerate(files[:300]):
    print (ix)
#    print(file)
#    print(gfiles[ix])
    
    try:
        gmisat = GMI_Sat(gfiles[ix])
    except:
        print("skipping this index, doing next file")
        continue
        

    
    
    dataset = xarray.open_dataset(file)
    
    LAT = np.concatenate([LAT, dataset.lat.data], axis = 0)
    LON = np.concatenate([LON, dataset.lon.data], axis = 0)
    IWP_mean = np.concatenate([IWP_mean, dataset.iwp_mean.data], axis = 0)
    IWP = np.concatenate([IWP, dataset.iwp.data], axis = 0)
    LSM = np.concatenate([LSM, dataset.stype.data], axis = 0)

    
    
    
    

    validation_data    = gmiSatData(gmisat, 
                             inputs, outputs,
                             batch_size = batchSize,
                             latlims = latlims)
    
    #lat = dataset.lat.data
    #lon = dataset.lon.data
    
    #lamask  = (lat > -65) & (lat < 65)
    #lomask  = (lon > 0) & (lon < 360)
    #dmask   = np.logical_and(lamask, lomask)
    #lsmmask = dataset.stype.data == 2
    
    #mask    = np.logical_and(lsmmask, dmask)
    #mask = dmask
    
    #if np.sum(mask) > 0:
        #IWP_mean  = np.concatenate([IWP_mean, dataset.iwp_mean.data[mask]], axis = 0)
    #TB   = np.concatenate([TB, validation_data.x[:, :, :4]], axis = 0)
        #Z0   = np.concatenate([Z0, validation_data.x[:, :, 7][mask]], axis = 0)
        #WVP  = np.concatenate([WVP, validation_data.x[:, :, 5][mask]], axis = 0)
    #else:
    #    continue
    





0
1
2
3
4
5


KeyboardInterrupt: 

In [ ]:
validation_data.x.shape

In [ ]:
d1 = xarray.Dataset({
    "iwp_mean": (["pixels", "scans"], IWP_mean),
    #"iwp0": (["pixels", "scans"], IWP0),
    "lsm": (["pixels", "scans"], LSM)} ,   
coords={
    "lon": (["pixels", "scans"], LON),
    "lat": (["pixels", "scans"], LAT),
})

d2 = xarray.Dataset({
    "tb": (["pixels", "scans", "channels"], TB)
})

In [ ]:
d1

In [ ]:
d2

In [ ]:
d2

outfile = os.path.expanduser("~/Dendrite/UserAreas/Kaur/jan2020_tb_lpa.nc")
d2.to_netcdf(outfile, mode = "w")

In [ ]:
outfile = os.path.expanduser("~/Dendrite/UserAreas/Kaur/jan2020_IWP_lpa_pr1.nc")
d1.to_netcdf(outfile, mode = "w")

In [ ]:
def histogram(iwp, bins):
    
    hist, _ = np.histogram(iwp, bins)
    
    return hist/np.sum(hist)


In [ ]:
bins = np.array([0.0,.0001,.00025,.0005,0.001,.0025,.005,.01,.025,.05,.1,.25,
                 .5,1, 2, 3, 4, 5, 6, 8, 10, 12, 16, 20, 24, 28, 32, 50])
    
bin_center = 0.5 * (bins[1:] + bins[:-1])   
latlims = [0, 30]
glat = d.lat.data

gmask = np.logical_and(np.abs(glat) >= latlims[0] , np.abs(glat) <= latlims[1])


ghist = histogram(d.iwp_mean.data[gmask], bins)



In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [8, 8])
ax.plot( bin_center, ghist, label = "adam_cosine")


ax.set_yscale("log")
ax.set_xscale("log")
ax.legend()

In [ ]:
def zonal_mean(lat, iwp, latbins):
    

    bins     = np.digitize(lat, latbins)
    
    nbins    = np.bincount(bins)
    iwp_mean = np.bincount(bins, iwp)
    
    return iwp_mean, nbins

In [ ]:
latbins = np.arange(-65, 66, 1.5)

#giwp = a.iwp_mean.data
giwp1 = d.iwp_mean.data
nanmask = giwp1 > -9000
#ziwp, ziwpc = zonal_mean(glat[nanmask], giwp[nanmask], latbins)
ziwp1, ziwpc1 = zonal_mean(glat[nanmask], giwp1[nanmask], latbins)

#plt.plot(ziwp[:-1]/ziwpc[:-1],latbins, 'b-',  label = "adam cosine")
plt.plot(ziwp1[:-1]/ziwpc1[:-1],latbins, 'r--',  label = "adam")
plt.legend()